In [35]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

In [2]:
file_path = '../Data/data_pre_procesada.csv'
data = pd.read_csv(file_path)
data.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil,Cta_Cliente_final,Temporada,Nueva_Coleccion
0,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW,2016-11-01
1,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK,china,2224074,FW,2016-11-01
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK,china,2224090,FW,2016-11-01
3,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK,china,2209051,FW,2016-11-01
4,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW,2016-11-01


In [3]:
# Agrupando por 'Cta_Cliente' y 'Familia'
grouped_data = data.groupby(['Cta_Cliente_final', 'familia'])['Tot_Piezas'].sum().reset_index()

grouped_data

,Cta_Cliente_final,familia,Tot_Piezas
0,1207097,4J,94.0
1,1207097,AF,581.0
2,1207097,AH,1903.0
3,1207097,BF,67.0
4,1207097,BH,825.0
...,...,...,...
811,2601001,TJ,5.0
812,2601001,WH,7.0
813,2601001,XH,3.0
814,2601001,YH,172.0


In [8]:
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(data[['Cta_Cliente_final', 'familia']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Cta_Cliente_final', 'familia']))
encoded_df['Tot_Piezas'] = data['Tot_Piezas']

/home/thayla/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
# variables independientes (X) y dependiente (y)
X = encoded_df.drop('Tot_Piezas', axis=1)
y = encoded_df['Tot_Piezas']
# Dividiendo los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Normalizar los datos de entrenamiento y de prueba
scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [36]:
# Arquitectura del modelo
model = Sequential([
    Dense(256, activation='relu', input_dim=X_train_scaled.shape[1]),
    BatchNormalization(),
    Dropout(0.2),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

model.fit(X_train_scaled, y_train, epochs=150, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/150
347/347 [==============================] - 3s 6ms/step - loss: 1177.3434 - val_loss: 788.4143
Epoch 2/150
347/347 [==============================] - 2s 6ms/step - loss: 886.9568 - val_loss: 625.2292
Epoch 3/150
347/347 [==============================] - 2s 6ms/step - loss: 841.5681 - val_loss: 620.1335
Epoch 4/150
347/347 [==============================] - 2s 5ms/step - loss: 843.5719 - val_loss: 625.5336
Epoch 5/150
347/347 [==============================] - 2s 5ms/step - loss: 845.9798 - val_loss: 621.8223
Epoch 6/150
347/347 [==============================] - 2s 6ms/step - loss: 832.8572 - val_loss: 623.7695
Epoch 7/150
347/347 [==============================] - 2s 5ms/step - loss: 823.8723 - val_loss: 646.5723
Epoch 8/150
347/347 [==============================] - 3s 8ms/step - loss: 825.0208 - val_loss: 655.3212
Epoch 9/150
347/347 [==============================] - 3s 8ms/step - loss: 825.0266 - val_loss: 637.1013
Epoch 10/150
347/347 [==============================] 

In [37]:
# y_pred son predicciones y y_test son los valores reales
y_pred = model.predict(X_test)

# Calcular las métricas
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")

217/217 [==============================] - 1s 2ms/step
MAE: 15.337830636985858
MSE: 1018.2339330514926
RMSE: 31.909778016330552
R-squared: 0.19293777853978544
